In this notebook we run a simple system of purely repulsive spheres using a [Weeks-Chandler-Andersen potential](https://hoomd-blue.readthedocs.io/en/stable/module-md-pair.html#hoomd.md.pair.lj). We visualize the spheres using the povray backend and color them by distance to their nearest neighbor.

In [ ]:
import flowws
import gtar
from hoomd_flowws.Init import Init
from hoomd_flowws.Interaction import Interaction
from hoomd_flowws.Run import Run

import plato, plato.draw.povray as draw
from plato.draw.vispy.internal import DEFAULT_DIRECTIONAL_LIGHTS as directional_lights
import freud
import numpy as np
import IPython

In [ ]:
storage = flowws.DirectoryStorage()
stages = [
    Init(number=128),
    Interaction(
        type='lj', global_params=[('r_cut', 2**(1./6))], 
        pair_params=[('A', 'A', 'epsilon', 1), ('A', 'A', 'sigma', 1)]),
    Run(steps=1e3, integrator='langevin'),
    Run(steps=1e4, integrator='langevin', compress_to=.57, dump_period=1e3),
]

flowws.Workflow(stages, storage).run()

In [ ]:
with gtar.GTAR('dump.sqlite', 'r') as traj:
    (posRec, boxRec), frames = traj.framesWithRecordsNamed(['position', 'box'])
    
    positions = traj.getRecord(posRec, frames[-1])
    box = traj.getRecord(boxRec, frames[-1])    

In [ ]:
# get nearest-neighbor distance, rescaled to go from 0-1, as cval
fbox = freud.box.Box.from_box(box)
nn = freud.locality.NearestNeighbors(1.5, 1).compute(fbox, positions, positions)
cval = nn.r_sq_list[:, 0].copy()
cval -= np.min(cval)
cval /= np.max(cval)

colors = plato.cmap.cubehelix(.25 + .5*cval)
prim = draw.Spheres(positions=positions, diameters=np.ones(len(positions)), colors=colors)
features = dict(ambient_light=.4, directional_light=directional_lights)
scene = draw.Scene(prim, features=features, zoom=4.8)

target = '../../gallery/flowws_spheres_povray.png'
scene.save(target)
IPython.display.Image(filename=target)